In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

!pip install split-folders
import splitfolders

print("Modules Imported Successfully")

In [ ]:
#Reduce data so that we can do the workshops quickly
#os.makedirs('DR/train')
#os.makedirs('DR/test')
#divide train data onto 8% train 2% test
initial_dataset = '../input/diabetic-retinopathy-balanced/content/Diabetic_Balanced_Data/train'
splitfolders.ratio(initial_dataset, output='DR', seed=1337, ratio=(0.08, 0.9, 0.02))
print("Directories Created Successfully")

In [ ]:
train_path = 'DR/train';
test_path = 'DR/test';
print("Number of Classes in test data: ", len(os.listdir(train_path)))
print("Number of Classes in test data: ", len(os.listdir(test_path)))

In [ ]:
clahe = cv2.createCLAHE(clipLimit = 5)#Contrast Limited Adaptive Histogram Equalization
img_name = '10010_right.jpeg'
img_normal = cv2.imread(train_path + "/0/" + img_name)
img_normal = cv2.resize(img_normal, (224,224))
img_normal = cv2.cvtColor(img_normal, cv2.COLOR_BGR2GRAY)
img_normal_clahe = clahe.apply(img_normal) + 30


img_name_1 = '10312_left.jpeg'
img_proliferative = cv2.imread(train_path + "/4/" + img_name_1)
img_proliferative = cv2.resize(img_proliferative, (224,224))
img_proliferative = cv2.cvtColor(img_proliferative, cv2.COLOR_BGR2GRAY)
img_proliferative_clahe = clahe.apply(img_proliferative) + 30




fig, axs = plt.subplots(2,2,figsize=(10,6))
axs[0,0].imshow(img_normal)
axs[0,0].set_title("NORMAL")
axs[0,1].imshow(img_normal_clahe)
axs[0,1].set_title("NORMAL_CLAHE")
axs[1,0].imshow(img_proliferative)
axs[1,0].set_title("PROLIFERATIVE")
axs[1,1].imshow(img_proliferative_clahe)
axs[1,1].set_title("PROLIFERATIVE_CLAHE");

In [ ]:
#Keras’ ImageDataGenerator class allows to implement a solution where images are loaded 
#and augmented in batch as the training progresses.
img_width , img_height = [224,224]
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.2,1.0],
    horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
image_batch, label_batch = next(iter(train_generator))

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(15):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n])
        #print(list(label_batch[n]))
        #print(list(label_batch[n]).index(1.0))
        if list(label_batch[n]).index(1) == 0:
            plt.title("Normal")
        elif list(label_batch[n]).index(1) == 1:
            plt.title("Mild")
        elif list(label_batch[n]).index(1) == 2:
            plt.title("Moderate")
        elif list(label_batch[n]).index(1) == 3:
            plt.title("Severe")
        else:
            plt.title("Proliferative")
        plt.axis("off")

show_batch(image_batch, label_batch)

In [ ]:
#nb_train_samples = 34792 # number of training samples
nb_train_samples = 2783
nb_test_samples = 697
#nb_test_samples = 4971 # number of training samples
epochs = 3  # number of epochs we gonna run
batch_size  = 32 # batch size ( at every iteration it will take 32 batches for training)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu", input_shape = (224,224,3)))
#32 filters 3x3 filter size
model.add(MaxPool2D())
#relu eliminates negative vals, max pooling reduces size of input

model.add(Conv2D(32, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))#avoid overfitting

model.add(Flatten()) 
model.add(Dense(128, activation="relu"))
model.add(Dense(5, activation="softmax"))
#softmax for alot of classes

model.summary()

In [ ]:
opt = Adam(lr=0.000001)
#Adam replaces gradient descent
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=7,
                              verbose=1,
                              mode='max',
                              baseline=None,
                              restore_best_weights=True)
#Reduce learning rate when a metric has stopped improving.
rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=3,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

In [ ]:
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = 3, validation_data = test_generator, callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
# its always a good practice to load the model after saving with the best epochs 
model = keras.models.load_model('./Model.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(test_generator.classes, predictions, 
                            target_names = ['No DR (Class 0)','Mild (Class 1)',
                                            'Moderate (Class 2)', 'Severe (Class 3)',
                                            'Proliferative DR (Class 4)']))


In [ ]:
#Transfer Learning Part

base_model = tf.keras.applications.MobileNetV2(input_shape = (224, 224, 3),
                                               include_top = False,
                                               weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalAveragePooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.summary()

In [ ]:
#training
opt = Adam(lr=0.000001)
#Adam replaces gradient descent
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model2.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)




In [ ]:
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = 3, validation_data = test_generator, callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
# its always a good practice to load the model after saving with the best epochs 
model = keras.models.load_model('./Model2.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(test_generator.classes, predictions, 
                            target_names = ['No DR (Class 0)','Mild (Class 1)',
                                            'Moderate (Class 2)', 'Severe (Class 3)',
                                            'Proliferative DR (Class 4)']))


In [ ]:
#Transfer Learning Part

base_model = tf.keras.applications.VGG16(input_shape = (224, 224, 3),
                                               include_top = False,
                                               weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalMaxPooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.summary()

In [ ]:
#training
opt = Adam(lr=0.000001)
#Adam replaces gradient descent
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model3.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
 

In [ ]:
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = 3, validation_data = test_generator, callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
# its always a good practice to load the model after saving with the best epochs 
model = keras.models.load_model('./Model3.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(test_generator.classes, predictions, 
                            target_names = ['No DR (Class 0)','Mild (Class 1)',
                                            'Moderate (Class 2)', 'Severe (Class 3)',
                                            'Proliferative DR (Class 4)']))


In [ ]:
#Transfer Learning Part

base_model = tf.keras.applications.VGG19(input_shape = (224, 224, 3),
                                               include_top = False,
                                               weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalMaxPooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.summary()

In [ ]:
#training
opt = Adam(lr=0.000001)
#Adam replaces gradient descent
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model4.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
 

In [ ]:
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = 3, validation_data = test_generator, callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
# its always a good practice to load the model after saving with the best epochs 
model = keras.models.load_model('./Model4.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(test_generator.classes, predictions, 
                            target_names = ['No DR (Class 0)','Mild (Class 1)',
                                            'Moderate (Class 2)', 'Severe (Class 3)',
                                            'Proliferative DR (Class 4)']))


In [ ]:
#Transfer Learning Part

base_model = tf.keras.applications.Xception(input_shape = (224, 224, 3),
                                               include_top = False,
                                               weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalMaxPooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalMaxPooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.summary()

In [ ]:
#training
opt = Adam(lr=0.000001)
#Adam replaces gradient descent
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model5.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
 

In [ ]:
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = 3, validation_data = test_generator, callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
# its always a good practice to load the model after saving with the best epochs 
model = keras.models.load_model('./Model5.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(test_generator.classes, predictions, 
                            target_names = ['No DR (Class 0)','Mild (Class 1)',
                                            'Moderate (Class 2)', 'Severe (Class 3)',
                                            'Proliferative DR (Class 4)']))


In [ ]:
#Transfer Learning Part

base_model = tf.keras.applications.InceptionV3(input_shape = (224, 224, 3),
                                               include_top = False,
                                               weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = tf.keras.Sequential([base_model,
                                 tf.keras.layers.GlobalMaxPooling2D(),
                                 tf.keras.layers.Dropout(0.2),
                                 tf.keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.summary()

In [ ]:
#training
opt = Adam(lr=0.001)
#Adam replaces gradient descent
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model6.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
 

In [ ]:
history = model.fit(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = 3, validation_data = test_generator, callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
# its always a good practice to load the model after saving with the best epochs 
model = keras.models.load_model('./Model6.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(test_generator.classes, predictions, 
                            target_names = ['No DR (Class 0)','Mild (Class 1)',
                                            'Moderate (Class 2)', 'Severe (Class 3)',
                                            'Proliferative DR (Class 4)']))

In [ ]:
def generate_image_adversary(model, image, label, eps=2 / 255.0):#model to fool, image to missclass nd label
    #eps small to fool comp not the human eye
    # cast the image
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tacked for
        # gradient updates
        tape.watch(image)
        # use our model to make predictions on the input image and then compute the loss
        pred = model.predict(image)
       # print(pred)
        loss = tf.keras.losses.CategoricalCrossentropy(label, pred)
        # calculate the gradients of loss with respect to the image, then
        # compute the sign of the gradient
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)
        # construct the image adversary
        adversary = (image + (signedGrad * eps)).numpy()
        # return the image adversary to the calling function
        return adversary

In [ ]:
# loop over a sample of our testing images
x,y = train_generator.next()
for i in np.random.choice(np.arange(0, len(x)), size=(10,)):
    # grab the current image and label
    image = x[i]
    #plt.imshow(image)
    #print(type(image))
    label = y[i].tolist().index(1)
    # generate an image adversary for the current image and make
    # a prediction on the adversary
    image = image.reshape(-1, 224, 224, 3)
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    adversary = generate_image_adversary(model,image, label, eps=0.1)
    pred = model.predict(adversary)
   